In [1]:
import json
import os
from glob import glob
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf

In [1]:
MODEL = "/kaggle/input/pp21models/A_baseline_noaug.fold_1"

paths = glob("/kaggle/input/**/test_images/*.jpg", recursive=True)
files = list(map(os.path.basename, paths))

model = tf.keras.models.load_model(f"{MODEL}.h5")
meta = json.loads(Path(f"{MODEL}.meta.json").read_text())

In [1]:
def _resize_tf_x2_2(path, size, zoom):

    # decode jpeg
    x = tf.io.read_file(path)
    x = tf.image.decode_jpeg(x, channels=3)

    # zoom=1
    x1 = tf.image.resize(x, size, method=tf.image.ResizeMethod.BICUBIC)
    x1 = x1.numpy().astype(np.uint8)

    # zoom=zoom
    x2 = tf.image.resize(
        x,
        (int(size[0] * zoom), int(size[1] * zoom)),
        method=tf.image.ResizeMethod.BICUBIC,
    )
    x2 = x2.numpy().astype(np.uint8)

    # center crop
    hor = (x2.shape[0] - size[0]) // 2
    ver = (x2.shape[1] - size[1]) // 2
    x2 = x2[ver:-ver, hor:-hor, :]

    assert x1.shape == size + (3,)
    assert x1.dtype == np.uint8

    assert x2.shape == size + (3,)
    assert x2.dtype == np.uint8

    return x1, x2


def _path_to_x(path):
    return _resize_tf_x2_2(
        path=path,
        size=meta["image_size"][:2],
        zoom=meta["agrs"]["zooms"][1],
    )


def _y_to_labels(y):
    labels = []

    for i, v in enumerate(y):
        if v > 0.5:
            labels.append(meta["classes"][i])

    if len(labels) == 0:
        labels = ["healthy"]

    return " ".join(labels)

In [1]:
labels = []

for path in paths:
    x1, x2 = _path_to_x(path)
    y = model.predict([np.array([x]), np.array([x])], verbose=0)[0]
    labels.append(_y_to_labels(y))

In [1]:
dfs = pd.DataFrame()
dfs.insert(0, "image", files)
dfs.insert(1, "labels", labels)
dfs.to_csv("/kaggle/working/submission.csv", index=False)

In [1]:
# ! cat /kaggle/working/submission.csv